In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim

import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler, random_split

from PIL import Image
import cv2
import os
import shutil
import time

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
print("We're using => ", device)

root_dir = ""
print("The data lies here => ", root_dir)
if not os.path.exists(root_dir+'train/'):
    os.makedirs(root_dir+'train/')
if not os.path.exists(root_dir+'test/'):
    os.makedirs(root_dir+'/test')
    
# model_dir = root_dir + 'model/'
# print("The best model lies here => ", model_dir)
# if not os.path.exists(model_dir):
#     os.makedirs(model_dir)
    
# ckp_dir = root_dir + 'checkpoint/'
# print("Lastest model checkpoint lies here => ", ckp_dir)
# if not os.path.exists(ckp_dir):
#     os.makedirs(ckp_dir)

We're using =>  cpu
The data lies here =>  


## Load data

In [3]:
# Define transforms
image_transforms = {
    # Train uses data augmentation
    'train':
    transforms.Compose([
        transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
        transforms.ColorJitter(),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(25),
#         transforms.RandomGrayscale(p=0.2),
        transforms.CenterCrop(size=224),  # Image net standards
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])  # Imagenet standards
    ]),
    # Validation does not use augmentation
    'test':
    transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

In [4]:
# Initialize Datasets
# Train data floders with be divied into train + val
product_dataset = datasets.ImageFolder(root=root_dir+"train",
                                      transform=image_transforms["train"])
class_names = product_dataset.classes
product_dataset

Dataset ImageFolder
    Number of datapoints: 105392
    Root location: train
    StandardTransform
Transform: Compose(
               RandomResizedCrop(size=(256, 256), scale=(0.8, 1.0), ratio=(0.75, 1.3333), interpolation=PIL.Image.BILINEAR)
               ColorJitter(brightness=None, contrast=None, saturation=None, hue=None)
               RandomHorizontalFlip(p=0.5)
               RandomRotation(degrees=(-25, 25), resample=False, expand=False)
               CenterCrop(size=(224, 224))
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

In [5]:
demo_size = 0.02 # 2K
demo_len = int(demo_size*len(product_dataset))
tiny_dataset, _ = random_split(product_dataset, [demo_len, len(product_dataset)-demo_len])
tiny_dataset

In [6]:
demo = False
if demo == True:
    print("we're using demo small dataset.")
    input_dataset = tiny_dataset
else:
    print("we're using normal big dataset")
    input_dataset = product_dataset

we're using normal big dataset


In [7]:
# Get train and validation samples
# SubsetRandomSampler
input_dataset_size = len(input_dataset)
input_dataset_indices = list(range(input_dataset_size))

np.random.shuffle(input_dataset_indices)
val_split_index = int(np.floor(0.2 * input_dataset_size))

train_idx, val_idx = input_dataset_indices[val_split_index:],\
input_dataset_indices[:val_split_index]

# train, val samplers
train_sampler = SubsetRandomSampler(train_idx)
val_sampler = SubsetRandomSampler(val_idx)
# train, val dataloaders
batch_size = 4
dataloaders = {
    "train": DataLoader(input_dataset, shuffle=False, 
                           batch_size=batch_size, sampler=train_sampler),
    "val": DataLoader(input_dataset, shuffle=False, 
                        batch_size=32, sampler=val_sampler)
}
dataset_sizes = {'train': input_dataset_size-val_split_index, 
                 'val': val_split_index}

In [8]:
dataset_sizes

{'train': 84314, 'val': 21078}

## Load model

In [9]:
#using efficientnet model based transfer learning
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        # freeze trained model
        self.model_ft = torch.load('pretrained_resnext.pt')
        # for param in self.pretrained.parameters():
        #   param.requires_grad = False
        self.l1 = nn.Linear(1000 , 256)
        self.dropout = nn.Dropout(0.75)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(256, 42)

    def forward(self, input):
        x = self.model_ft(input)
        x = x.view(x.size(0),-1)
        x = self.dropout(self.relu(self.l1(x)))
        x = self.l2(x)
        return x

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_ft = Classifier().to(device)

# check our model is on right device
print("Our classifier is on the device => ", list(model_ft.parameters())[0].device)

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:453: SourceChangeWarning: source code of class 'torchvision.models.resnet.ResNet' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:453: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:453: SourceChangeWarning: source code of class 'torch.nn.modules.activation.ReLU' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torc

Our classifier is on the device =>  cpu


In [10]:
def save_ckp(state, is_best, checkpoint_path, best_model_path):
    """
    state: checkpoint we want to save
    is_best: is this the best checkpoint; min validation loss
    checkpoint_path: path to save checkpoint
    best_model_path: path to save best model
    """
    f_path = checkpoint_path
    # save checkpoint data to the path given, checkpoint_path
    torch.save(state, f_path)
    # if it is a best model, min validation loss
    if is_best:
        best_fpath = best_model_path
        # copy that checkpoint file to best path given, best_model_path
        shutil.copyfile(f_path, best_fpath)

In [11]:
def load_ckp(checkpoint_fpath, model):
    """
    checkpoint_path: path to save checkpoint
    model: model that we want to load checkpoint parameters into       
    optimizer: optimizer we defined in previous training
    """
    # load check point
    checkpoint = torch.load(checkpoint_fpath)
    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['state_dict'])
    # initialize optimizer from checkpoint to optimizer
    # optimizer.load_state_dict(checkpoint['optimizer'])
    # initialize valid_loss_min from checkpoint to valid_loss_min
    valid_loss_min = checkpoint['valid_loss_min']
    # initialize valid_acc_best from checkpoint to valid_acc_best
    valid_acc_best = checkpoint['valid_acc_best']
    # return model, optimizer, epoch value, min validation loss 
    return model, checkpoint['epoch'], valid_loss_min, valid_acc_best.item()

In [12]:
!nvidia-smi

Sat Jul  4 09:26:39 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:3E:00.0 Off |                    0 |
| N/A   54C    P0   249W / 280W |   8505MiB / 16160MiB |     78%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [32]:
# model_ft = models.resnext101_32x8d(pretrained=True, progress=True)
# torch.save(model_ft, model_dir+"best_model_11pm.pt")
model_ft, epoch, loss_min, acc_best = load_ckp("model/best_model.pt", model_ft)
torch.cuda.empty_cache()

In [33]:
!nvidia-smi

Sat Jul  4 10:05:20 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:3E:00.0 Off |                    0 |
| N/A   53C    P0    68W / 280W |   9706MiB / 16160MiB |     52%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [34]:
print("start_epoch = ", epoch)
# print("valid_loss_min = ", valid_loss_min)
print("valid_loss_min = {:.6f}".format(loss_min))
print("valid_acc_best = ", acc_best)

start_epoch =  7
valid_loss_min = 0.433545
valid_acc_best =  0.8794952082740298


## Val acc by every category

In [35]:
criterion = nn.CrossEntropyLoss()
# observe that all parameters are being optimized
optimizer = optim.Adam(model_ft.parameters(), lr=0.0001)
# Decay LR by a factor of 0.1 every 5 epochs
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

In [36]:
!nvidia-smi

Sat Jul  4 10:19:03 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:3E:00.0 Off |                    0 |
| N/A   55C    P0   244W / 280W |   9706MiB / 16160MiB |     53%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [37]:
def eval_model(model, data_loader, criterion, optimizer):
    since = time.time()
    torch.cuda.empty_cache()
    # Each epoch has a training and validation phase
    !nvidia-smi
    model.eval()   # Set model to evaluate mode
    running_loss = 0.0
    running_corrects = 0
    CNT = 0
    phase = 'val'
    label_list = []
    pred_list = []
    # Iterate over data.
    for batch_idx, (inputs, labels) in enumerate(data_loader):
        with torch.no_grad():
            inputs = inputs.to(device)
            labels = labels.to(device)
            # zero the parameter gradients
            optimizer.zero_grad()
            # forward
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            _, preds = torch.max(outputs, 1)
        # record
        label_list.extend(labels.tolist())
        pred_list.extend(preds.tolist())
        # statistics
        batchsize = inputs.size(0)
        running_loss += loss.item() * batchsize
        running_corrects += torch.sum(preds == labels.data)
        CNT += 1
        if batch_idx % 100 == 0:
            print('   {}:  [{}/{} ({:.0f}%)]\tLoss: {:.6f} \tAcc: {:.6f}'.format(phase,
                CNT*batchsize, dataset_sizes[phase], 100. * CNT/len(data_loader),
                running_loss/(CNT*batchsize), running_corrects.double()/(CNT*batchsize)))
        # demo for program checking
        if batch_idx == 100:
            break

    epoch_loss = running_loss / dataset_sizes['val']
    epoch_acc = running_corrects.double() / dataset_sizes['val']

    print('{} Loss: {:.4f} Acc: {:.4f}'.format(
        phase, epoch_loss, epoch_acc))

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Average val Acc: {:4f}'.format(epoch_acc))

    res = pd.DataFrame({'label': label_list, 'pred': pred_list})
    return res

In [38]:
val_acc = eval_model(model_ft, dataloaders['val'], criterion, optimizer)

Sat Jul  4 10:19:06 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:3E:00.0 Off |                    0 |
| N/A   52C    P0   209W / 280W |   9706MiB / 16160MiB |      4%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [39]:
val_acc['T&F'] = np.where(val_acc['label']==val_acc['pred'], 1, 0)
val_acc

,label,pred,T&F
0,0,0,1
1,27,27,1
2,10,10,1
3,34,34,1
4,36,36,1
...,...,...,...
3227,0,0,1
3228,4,4,1
3229,28,28,1
3230,24,24,1


In [40]:
category_acc = val_acc[['label', 'T&F']].groupby('label').agg(
    {'T&F': ['count', 'sum']})
category_acc.columns = category_acc.columns.droplevel(0)
#pandas 0.18.0 and higher
category_acc = category_acc.rename_axis(None, axis=1)
category_acc['accuracy'] = category_acc['sum'] / category_acc['count']
category_acc.style.background_gradient()

,count,sum,accuracy
label,,,
0,93,60,0.645161
1,70,60,0.857143
2,69,58,0.84058
3,97,91,0.938144
4,73,67,0.917808
5,89,84,0.94382
6,71,68,0.957746
7,91,86,0.945055
8,83,80,0.963855


## Make prediction

In [41]:
# make prediction on test
class myTestData(Dataset):
    def __init__(self, base_path, transform=None):
        f = pd.read_csv(base_path+"test.csv")
        self.labels = [0]*len(f)
        self.img_folder = os.path.join(base_path,"test/")
        self.filenames = f.filename.to_list()
        assert len(self.labels) == len(self.filenames)
        num_data = len(self.filenames)
        self.transform = transform

    def __getitem__(self, idx):
        img_name = self.filenames[idx]
        img_path = self.img_folder + img_name
        label = self.labels[idx]
        # print(os.path.join(self.output_path, path))
        # 
        img = Image.open(img_path)
        data = np.asarray(img)
#         print(data.shape)
        if self.transform(img) is not None:
            img = self.transform(img)
#         print(img.shape)
        return img_name, img, label

    def __len__(self):
        return len(self.filenames)

test_data = myTestData(base_path=root_dir, transform=image_transforms['test'])
test_loader = DataLoader(test_data, batch_size=4, shuffle=False)

In [42]:
test_data[0]

('fd663cf2b6e1d7b02938c6aaae0a32d2.jpg',
 tensor([[[-0.6109, -0.8507, -0.9020,  ..., -0.0629, -0.0629, -0.1828],
          [-0.8507, -0.9705, -0.9534,  ..., -0.0972, -0.1143, -0.1828],
          [-0.9363, -0.9705, -0.8507,  ..., -0.1657, -0.1999, -0.1828],
          ...,
          [-0.5082, -0.3369, -0.3883,  ..., -0.9705, -0.9705, -0.9363],
          [-0.5424, -0.1314, -0.0116,  ..., -1.0048, -0.9705, -0.9020],
          [-0.3541, -0.5424, -0.4568,  ..., -0.9534, -0.9192, -0.8678]],
 
         [[-0.7577, -1.0378, -1.1078,  ...,  0.0651,  0.0651, -0.1099],
          [-1.0553, -1.2129, -1.2129,  ...,  0.0651,  0.0476, -0.0574],
          [-1.1779, -1.2304, -1.1253,  ...,  0.0651,  0.0301,  0.0301],
          ...,
          [-0.6877, -0.5126, -0.5476,  ..., -1.1779, -1.1954, -1.1954],
          [-0.7227, -0.3025, -0.1800,  ..., -1.2129, -1.1954, -1.1604],
          [-0.5301, -0.7227, -0.6352,  ..., -1.1604, -1.1604, -1.1253]],
 
         [[-0.1312, -0.3753, -0.4275,  ...,  0.1476,  0.165

In [43]:
test_size = len(test_data)

In [44]:
def model_pred(model, data_loader, criterion, optimizer, data_size):
    since = time.time()
    torch.cuda.empty_cache()
    # Each epoch has a training and validation phase
    model.eval()   # Set model to evaluate mode
    running_loss = 0.0
    running_corrects = 0
    CNT = 0
    phase = 'test'
    name_list = []
    pred_list = []
    # Iterate over data.
    for batch_idx, (fnames, inputs, labels) in enumerate(data_loader): 
#         !nvidia-smi
        with torch.no_grad():
            inputs = inputs.to(device)
            labels = labels.to(device)
            # zero the parameter gradients
            optimizer.zero_grad()
            # forward
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            _, preds = torch.max(outputs, 1)
        # record
        name_list.extend(list(fnames))
        pred_list.extend(preds.tolist())
        # statistics
        batchsize = inputs.size(0)
        running_loss += loss.item() * batchsize
        running_corrects += torch.sum(preds == labels.data)
        CNT += 1
        if batch_idx % 100 == 0:
            print('   {}:  [{}/{} ({:.0f}%)]\tLoss: {:.6f} \tAcc: {:.6f}'.format(phase,
                CNT*batchsize, data_size, 100. * CNT/len(data_loader),
                running_loss/(CNT*batchsize), running_corrects.double()/(CNT*batchsize)))
        
        # demo for program checking
#         if batch_idx == 100:
#             break

    epoch_loss = running_loss / data_size
    epoch_acc = running_corrects.double() / data_size

    print('{} Loss: {:.4f} Acc: {:.4f}'.format(
        phase, epoch_loss, epoch_acc))

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Average val Acc: {:4f}'.format(epoch_acc))

    res = pd.DataFrame({'filename': name_list, 'category': pred_list})
    return res

In [45]:
submission = model_pred(model_ft, test_loader, criterion, optimizer, test_size)

   test:  [4/12186 (0%)]	Loss: 19.434780 	Acc: 0.000000
   test:  [404/12186 (3%)]	Loss: 20.893785 	Acc: 0.034653
   test:  [804/12186 (7%)]	Loss: 21.006585 	Acc: 0.027363
   test:  [1204/12186 (10%)]	Loss: 20.871162 	Acc: 0.029900
   test:  [1604/12186 (13%)]	Loss: 20.716096 	Acc: 0.033666
   test:  [2004/12186 (16%)]	Loss: 20.545782 	Acc: 0.031437
   test:  [2404/12186 (20%)]	Loss: 20.645128 	Acc: 0.028702
   test:  [2804/12186 (23%)]	Loss: 20.640648 	Acc: 0.028174
   test:  [3204/12186 (26%)]	Loss: 20.729433 	Acc: 0.026841
   test:  [3604/12186 (30%)]	Loss: 20.677492 	Acc: 0.026637
   test:  [4004/12186 (33%)]	Loss: 20.730950 	Acc: 0.025724
   test:  [4404/12186 (36%)]	Loss: 20.771605 	Acc: 0.024977
   test:  [4804/12186 (39%)]	Loss: 20.745860 	Acc: 0.025812
   test:  [5204/12186 (43%)]	Loss: 20.784826 	Acc: 0.025557
   test:  [5604/12186 (46%)]	Loss: 20.736716 	Acc: 0.025339
   test:  [6004/12186 (49%)]	Loss: 20.782045 	Acc: 0.025150
   test:  [6404/12186 (53%)]	Loss: 20.808328 	Ac

In [46]:
submission.category = submission.category.apply(lambda x: class_names[x])

In [47]:
submission

,filename,category
0,fd663cf2b6e1d7b02938c6aaae0a32d2.jpg,20
1,c7fd77508a8c355eaab0d4e10efd6b15.jpg,27
2,127f3e6d6e3491b2459812353f33a913.jpg,04
3,5ca4f2da11eda083064e6c36f37eeb81.jpg,22
4,46d681a542f2c71be017eef6aae23313.jpg,12
...,...,...
12181,5ba958eacb23cd7d1673bad4dae55784.jpg,16
12182,efbe41a1c2b666b70e337e438559808b.jpg,19
12183,79fdaa5ac5ba10dbe8004cabd8c35eb3.jpg,13
12184,ac3d136124617637a05ba66694e381ef.jpg,15


In [48]:
submission.to_csv('submission.csv', index=False)

In [30]:
test = val_acc[val_acc.label==0]

In [31]:
test.groupby('pred').size()

pred
0     57
1     10
2      3
3      3
6      1
17     1
dtype: int64